# Fitting V4 voxels from Alexnet conv3 features - take #2
Here we're trying again to find a mapping from the alexnet conv3 feature space to the V4 voxel space. But this time the features come from 'retinawarp'ed' input images

In [2]:
#basic imports
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pymongo as pm
import numpy as np
import h5py
import scipy as sp
import scipy.stats as stats
import os

from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import RidgeCV, Ridge

In [3]:
# load and organize brain data

#read in the mat files
with h5py.File('data/EstimatedResponses.mat','r') as fmri_dataset:
    train_S1 = fmri_dataset['dataTrnS1'][:]
    test_S1 = fmri_dataset['dataValS1'][:]
    roi_S1 = fmri_dataset['roiS1'][:]
    
    train_S2 = fmri_dataset['dataTrnS2'][:]
    test_S2 = fmri_dataset['dataValS2'][:]
    roi_S2 = fmri_dataset['roiS2'][:]
    
    unique_ROIs = np.unique((fmri_dataset['roiS1'])) 
    
#organize subject 1
S1_train_data_by_ROI = {c: train_S1[:,roi_S1[0,:] == c] 
                       for c in unique_ROIs}
S1_test_data_by_ROI = {c: test_S1[:,roi_S1[0,:] == c] 
                       for c in unique_ROIs}
#organize subject 2
S2_train_data_by_ROI = {c: train_S2[:,roi_S2[0,:] == c] 
                       for c in unique_ROIs}
S2_test_data_by_ROI = {c: test_S2[:,roi_S2[0,:] == c] 
                       for c in unique_ROIs}

#Select only V4 voxels & get rid of any voxels that have NaNs for any training images
#Subject 1
S1_V4 = S1_train_data_by_ROI[6.0].T
x=S1_V4[~np.isnan(S1_V4).any(axis=1)]
S1_V4_train = x.T
S1_V4_val = S1_test_data_by_ROI[6.0].T
x=S1_V4_val[~np.isnan(S1_V4).any(axis=1)]
S1_V4_test = x.T
#Subject 2
S2_V4 = S2_train_data_by_ROI[6.0].T
x=S2_V4[~np.isnan(S2_V4).any(axis=1)]
S2_V4_train = x.T
S2_V4_val = S2_test_data_by_ROI[6.0].T
x=S2_V4_val[~np.isnan(S2_V4).any(axis=1)]
S2_V4_test = x.T
sum(sum(np.isnan(S2_V4_test))) #make sure we don't have any NaNs left in the test data

0

In [ ]:
# read in the alexnet features
h5f = h5py.File('alexnet/alexnet_conv3_features_retinawarp.h5','r')
train_features = h5f['train'][:] #features for the training images
val_features = h5f['val'][:] #features for the validation images
h5f.close()